In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

c:\Users\Angelika Vergara\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load datasets
kwaai_data = pd.read_csv('train.csv')
pt_sk_data = pd.read_csv('toxic_conversations_balanced.csv')
data = pd.concat([kwaai_data, pt_sk_data], ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True)

In [3]:
# Text preprocessing
def clean_text(text):
    text = re.sub(r"http\\S+", "", text)
    text = re.sub(r"[^a-zA-Z]", " ", text)
    return text.lower()

data['text'] = data['text'].apply(clean_text)


In [4]:
# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['text'].tolist(), data['label'].tolist(), test_size=0.2, random_state=42
)

In [5]:
# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [7]:
# Create dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }


In [8]:
# Create data loaders
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
test_dataset = TextDataset(test_texts, test_labels, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [9]:
# Define model
class BertLSTMClassifier(nn.Module):
    def __init__(self, bert_model='bert-base-uncased', hidden_dim=128, num_classes=1):
        super(BertLSTMClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model)
        self.lstm = nn.LSTM(768, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.dropout = nn.Dropout(0.3)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask):
        with torch.no_grad():  # Freeze BERT
            bert_output = self.bert(input_ids, attention_mask=attention_mask)
        lstm_output, _ = self.lstm(bert_output.last_hidden_state)
        lstm_output = lstm_output[:, -1, :]
        output = self.fc(self.dropout(lstm_output))
        return self.sigmoid(output)

In [12]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Define your model (example: a simple LSTM-BERT hybrid)
class SentimentModel(nn.Module):
    def __init__(self, bert_model):
        super(SentimentModel, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(p=0.3)  # Prevent overfitting
        self.fc = nn.Linear(768, 1)  # Adjust based on hidden size

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token output
        x = self.dropout(pooled_output)
        x = self.fc(x)
        return x

# Load pretrained BERT (Example: "bert-base-uncased")
from transformers import BertModel
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Initialize model
model = SentimentModel(bert_model).to(device)  # Move model to GPU if available


In [1]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")
print("CUDA Version:", torch.version.cuda)
print("PyTorch Version:", torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", device)


CUDA Available: True
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU
CUDA Version: 12.1
PyTorch Version: 2.5.1+cu121
Using Device: cuda


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", device)


Using Device: cuda


In [2]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Define your model (example: a simple LSTM-BERT hybrid)
class SentimentModel(nn.Module):
    def __init__(self, bert_model):
        super(SentimentModel, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(p=0.3)  # Prevent overfitting
        self.fc = nn.Linear(768, 1)  # Adjust based on hidden size

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token output
        x = self.dropout(pooled_output)
        x = self.fc(x)
        return x

# Load pretrained BERT (Example: "bert-base-uncased")
from transformers import BertModel
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Initialize model
model = SentimentModel(bert_model).to(device)  # Move model to GPU if available


c:\Users\Angelika Vergara\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔥 Using device: {device}")

# Define Loss and Optimizer with L2 Regularization
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)  # L2 Regularization

# Learning Rate Scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

# Early Stopping
best_val_loss = float('inf')
early_stopping_patience = 3
early_stopping_counter = 0

# Training Loop
epochs = 10  # Adjust as needed
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(epochs):
    model.train()
    epoch_loss, epoch_accuracy = 0, 0
    num_batches = 0

    for batch in train_loader:
        num_batches += 1
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        predictions = (torch.sigmoid(outputs) > 0.5).float()
        batch_accuracy = (predictions == labels).float().mean().item()
        epoch_accuracy += batch_accuracy

    avg_train_loss = epoch_loss / num_batches
    avg_train_acc = epoch_accuracy / num_batches
    train_losses.append(avg_train_loss)
    train_accuracies.append(avg_train_acc)

    # Validation Loop
    model.eval()
    val_loss, val_accuracy = 0, 0
    num_val_batches = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            num_val_batches += 1
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            predictions = (torch.sigmoid(outputs) > 0.5).float()
            batch_accuracy = (predictions == labels).float().mean().item()
            val_accuracy += batch_accuracy

            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / num_val_batches
    avg_val_acc = val_accuracy / num_val_batches
    val_losses.append(avg_val_loss)
    val_accuracies.append(avg_val_acc)

    # Compute Precision, Recall, and F1-Score
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}, "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}, Precision: {precision:.4f}, "
          f"Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    # Learning Rate Scheduler
    scheduler.step(avg_val_loss)

    # Early Stopping Check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stopping_counter = 0  # Reset counter
        torch.save(model.state_dict(), "best_model.pth")  # Save best model
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print("⏹️ Early stopping triggered!")
            break

print(f"✅ Training complete! Best Validation Loss: {best_val_loss:.4f}")


🔥 Using device: cuda


NameError: name 'train_loader' is not defined